In [1]:
import torch
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset_path = './trainingData'
dataset = ImageFolder(root=dataset_path, transform=transform)

# Split dataset into training, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Extract features from images
def extract_features_from_loader(loader, model):
    features = []
    labels = []
    model.eval()
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            features.extend(outputs.cpu().numpy())
            labels.extend(targets.cpu().numpy())
    return np.array(features), np.array(labels)

# Define a simple feature extraction model
class FeatureExtractor(torch.nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.pool = torch.nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        return x

# Initialize feature extractor
feature_extractor = FeatureExtractor()

# Extract features for training data
train_features, train_labels = extract_features_from_loader(train_dataloader, feature_extractor)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)

# Extract features for validation data
val_features, val_labels = extract_features_from_loader(val_dataloader, feature_extractor)

# Predict and compute accuracy for validation data
val_predictions = svm_classifier.predict(val_features)
val_accuracy = accuracy_score(val_labels, val_predictions)
print(f'Validation Accuracy using SVM: {val_accuracy:.4f}')

# Extract features for test data
test_features, test_labels = extract_features_from_loader(test_dataloader, feature_extractor)

# Predict and compute accuracy for test data
test_predictions = svm_classifier.predict(test_features)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test Accuracy using SVM: {test_accuracy:.4f}')

# Validation Accuracy using SVM: 0.9015
# Test Accuracy using SVM: 0.9356
# in 2 minutes

Validation Accuracy using SVM: 0.9015
Test Accuracy using SVM: 0.9356
